<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/TwADR_bigtweet_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'TwADR-L.fold-7.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"TwADR-L.fold-7.validation.csv",header=None)
valid.head()

,0,1
0,13,insomniac
1,24,racing heart
2,60,cigarette withdrawal
3,91,jitters
4,93,shooting


In [12]:
# testing data set

test = pd.read_csv(path/"TwADR-L.fold-7.test.csv",header=None)
test.head()

,0,1
0,13,keeping me awake
1,94,anxiety
2,119,incr anxiety
3,119,anxiety
4,121,itching


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'TwADR-bigtweet.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'to',
 'the',
 'in',
 'rt',
 'of',
 'for',
 'a',
 'you',
 'i',
 'is',
 'on',
 'disease',
 'my',
 'pain',
 'and',
 'mood',
 'me',
 'amp',
 'increased',
 'with',
 'gluten',
 ',',
 'disorder',
 'abnormal',
 'withdrawal',
 'anxiety',
 'your',
 'free',
 'it',
 'have',
 'symptoms',
 'weight',
 'this',
 'out',
 'are',
 'new',
 'decreased',
 'from',
 'not',
 'do',
 'about',
 'disorders',
 'like',
 '(',
 ')',
 'glutenfree',
 '-',
 'nt',
 'our',
 'up',
 'be',
 'what',
 'behavior',
 'drug',
 'sleeplessness',
 'syndrome',
 'celiac',
 'got',
 'at',
 'now',
 'feeling',
 'diseases',
 'or',
 'somnolence',
 'insomnia',
 'finding',
 'blood',
 'disturbance',
 'that',
 'sleep',
 'energy',
 'all',
 'we',
 'so',
 'mental',
 'right',
 'how',
 'food',
 'can',
 'depression',
 'gain',
 'appetite',
 'more',
 'acute',
 'skin',
 'feel',
 'via',
 'tip',
 'if',
 'its',
 'nausea',
 'diet',
 'irritable',
 'body',
 'ceiling',
 'a

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.369297,3.511884,0.455148,09:31


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.291300,2.995682,0.513825,09:31


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.035456,2.947534,0.518778,09:27


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.980360,2.907604,0.521903,09:35


In [25]:
learn.save_encoder('TwADR-bigtweet_fold7_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('TwADR-bigtweet_fold7_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.280585,6.281684,0.126761,00:28


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.263478,5.818071,0.161972,00:31


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.761266,4.973797,0.161972,01:09


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.476041,4.676027,0.211268,01:11
1,5.256366,4.565952,0.204225,01:09


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.004102,4.346807,0.274648,01:07
1,4.821793,4.222142,0.274648,01:09


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.589628,4.087850,0.295775,01:06
1,4.421877,3.960006,0.316901,01:10


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.225294,3.784171,0.323944,01:04
1,4.073627,3.705823,0.366197,01:11


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.889095,3.613324,0.345070,01:03
1,3.715437,3.537848,0.366197,01:06


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.593689,3.458236,0.366197,01:02
1,3.471777,3.389558,0.408451,01:10
2,3.321856,3.386578,0.415493,01:08
3,3.120277,3.359770,0.415493,01:06


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.935599,3.305051,0.401408,01:10
1,2.887062,3.270604,0.408451,01:10
2,2.719422,3.300037,0.387324,01:09
3,2.556783,3.267018,0.401408,01:07


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.423056,3.296239,0.408451,01:04
1,2.365986,3.288677,0.415493,01:08
2,2.195576,3.219337,0.401408,01:07
3,2.074013,3.208453,0.408451,01:09


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.919234,3.250190,0.422535,01:07
1,1.919992,3.215484,0.408451,01:05
2,1.798878,3.332634,0.415493,01:06
3,1.660861,3.322710,0.415493,01:10


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.523316,3.299520,0.408451,01:09
1,1.536404,3.434026,0.422535,01:10
2,1.404883,3.354916,0.436620,01:09
3,1.299919,3.360826,0.436620,01:10


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.181112,3.348787,0.429577,01:04
1,1.212193,3.454866,0.415493,01:06
2,1.137413,3.447193,0.436620,01:07
3,1.032148,3.439932,0.436620,01:04


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.976637,3.439493,0.436620,01:12
1,0.993702,3.522117,0.429577,01:05
2,0.928338,3.531862,0.429577,01:07
3,0.855353,3.516958,0.443662,01:05


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.796547,3.783062,0.429577,01:08
1,0.772732,3.716344,0.422535,01:04


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.734167,3.805909,0.422535,01:03
1,0.703224,3.770158,0.422535,01:03


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.689642,3.708044,0.429577,01:04


In [44]:
# save the best model

learn.save_encoder('TwADR-bigtweet_fold7')

In [45]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.640719,3.776748,0.429577,01:09


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.645979,3.744063,0.457746,01:06


In [47]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.612290,3.889948,0.436620,01:09


In [48]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.566485,3.967842,0.394366,01:05


In [49]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.588486,3.896019,0.443662,01:09


# Part three: Predict on the test dataset

In [50]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [51]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

142
142
[393, 978, 978, 978, 121, 32, 154, 1099, 1295, 189, 1099, 649, 231, 231, 1936, 2147, 2147, 2147, 1936, 1936, 249, 857, 300, 1054, 1099, 1610, 418, 426, 187, 596, 1787, 749, 1767, 2047, 749, 933, 749, 1597, 2032, 2032, 233, 824, 2168, 1884, 852, 1669, 978, 2149, 974, 792, 1062, 441, 978, 978, 1530, 74, 426, 1718, 1669, 1099, 1136, 1175, 1265, 852, 249, 1099, 121, 1308, 1884, 355, 2149, 1507, 1096, 974, 1432, 1576, 1701, 1701, 1099, 1573, 1594, 1597, 791, 1597, 1597, 1597, 1597, 1597, 1597, 1597, 1610, 1610, 265, 1099, 1727, 1099, 1531, 1767, 1767, 1767, 1767, 1099, 1767, 1767, 1767, 561, 1769, 1787, 2032, 1727, 2191, 1787, 1597, 418, 2191, 418, 418, 1915, 1936, 1936, 1936, 712, 1915, 1283, 1283, 1942, 2113, 16, 1265, 1325, 1813, 224, 418, 1769, 1545, 2119, 1545, 2147, 1099, 2191, 441, 1701]
[13, 94, 119, 119, 121, 143, 154, 189, 189, 189, 224, 230, 231, 231, 233, 233, 233, 233, 233, 233, 249, 265, 300, 359, 393, 396, 418, 426, 518, 596, 693, 712, 744, 749, 749, 749, 749, 791, 81

In [52]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

47
0.33098591549295775
